# Bernstein-Vazirani Problem: Find Hidden String s
# By Vivek Rajagopalan | Date: 24Dec2025 

---
### **What We’ll Learn**
Find a hidden binary string `s` using quantum parallelism. The function `f(x) = (x·s) mod 2` (dot product mod 2).

---
### **Problem Setup**
`x` is an `n`-bit input, `s` is an `n`-bit secret string. `f(x)` returns 1 if the sum of `(x_i * s_i)` is odd, else 0.
Example: If `s=101`, `x=011` → (0*1)+(1*0)+(1*1)=1 → `f(x)=1`.

**Goal**: Find `s` with minimal queries.

---
### **Classical Approach**
Need `n` queries (test x=001, 010, 100, etc.). Quantum does it in **1 query**!


---
### **Quantum Circuit**
Let’s use `n=3` to find `s=101`.

In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram

def bv_oracle(s, n):
    # s is the hidden string (e.g., '101')
    # Oracle computes f(x) = (x·s) mod 2 → entangles x with s
    qc = QuantumCircuit(n+1)
    # Initialize output qubit to |1>
    qc.x(n)
    qc.h(n)
    # Apply controlled-Z gates based on s
    for i in range(n):
        if s[i] == '1':
            qc.cz(i, n)
    qc.h(n)
    return qc

def quantum_bv(n, s):
    qc = QuantumCircuit(n+1, n)

    # Step 1: Superposition all input qubits
    for i in range(n):
        qc.h(i)

    # Step 2: Apply oracle (hidden s)
    oracle = bv_oracle(s, n)
    qc = qc.compose(oracle)

    # Step 3: Hadamard on input qubits again, then measure
    for i in range(n):
        qc.h(i)
        qc.measure(i, i)

    return qc

In [ ]:
# Let's test with s=101 (n=3)
n = 3
s = '101'
qc_bv = quantum_bv(n, s)
print("Quantum Circuit for s=101:")
qc_bv.draw(output='mpl')

simulator = Aer.get_backend('qasm_simulator')
result = execute(qc_bv, simulator, shots=1024).result()
counts = result.get_counts()
print("Measurement Results:")
plot_histogram(counts)